<a href="https://colab.research.google.com/github/JamesJokDutAkuei/Malaria_Diagnosis_Transfer_Learning/blob/main/JamesJokDutAkuei_Malaria_Diagnosis_CNN_Group6_EvenNumber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning for Malaria Diagnosis
This notebook is inspired by works of (Sivaramakrishnan Rajaraman  et al., 2018) and (Jason Brownlee, 2019). Acknowledge to NIH and Bangalor Hospital who make available this malaria dataset.

Malaria is an infectuous disease caused by parasites that are transmitted to people through the bites of infected female Anopheles mosquitoes.

The Malaria burden with some key figures:
<font color='red'>
* More than 219 million cases
* Over 430 000 deaths in 2017 (Mostly: children & pregnants)
* 80% in 15 countries of Africa & India
  </font>

![MalariaBurd](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/MalariaBurden.png?raw=1)

The malaria diagnosis is performed using blood test:
* Collect patient blood smear
* Microscopic visualisation of the parasit

![MalariaDiag](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/MalariaDiag.png?raw=1)
  
Main issues related to traditional diagnosis:
<font color='#ed7d31'>
* resource-constrained regions
* time needed and delays
* diagnosis accuracy and cost
</font>

The objective of this notebook is to apply modern deep learning techniques to perform medical image analysis for malaria diagnosis.

*This notebook is inspired by works of (Sivaramakrishnan Rajaraman  et al., 2018), (Adrian Rosebrock, 2018) and (Jason Brownlee, 2019)*

## Configuration

In [ ]:
#Mount the local drive project_forder
from google.colab import drive
drive.mount('/content/drive/')
!ls "/content/drive/My Drive/Colab Notebooks/10xDS/Projects/malaria-diagnosis/"

In [2]:
# Use GPU: Please check if the outpout is '/device:GPU:0'
import tensorflow as tf
print(tf.__version__)
tf.test.gpu_device_name()
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

2.19.0


'/device:GPU:0'

## Populating namespaces

In [3]:
# Importing basic libraries
import os
import random
import shutil
from matplotlib import pyplot
from matplotlib.image import imread
%matplotlib inline

# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D as Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [4]:
# Define the useful paths for data accessibility
ai_project = '.' #"/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
cell_images_dir = os.path.join(ai_project,'cell_images')
training_path = os.path.join(ai_project,'train')
testing_path = os.path.join(ai_project,'test')

## Prepare DataSet

### *Download* DataSet

In [5]:
# Download the data in the allocated google cloud-server. If already down, turn downloadData=False
downloadData = True
if downloadData == True:
  indrive = False
  if indrive == True:
    !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip -P "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
    !unzip "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis/cell_images.zip" -d "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis/"
    !ls "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
  else: #incloud google server
    !rm -rf cell_images.*
    !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
    !unzip cell_images.zip >/dev/null 2>&1
    !ls

--2025-10-05 15:15:54--  https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
Resolving data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)... 3.167.192.56, 3.167.192.60, 3.167.192.111, ...
Connecting to data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)|3.167.192.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353452851 (337M) [application/zip]
Saving to: ‘cell_images.zip’

cell_images.zip     100%[===================>] 337.08M   333MB/s    in 1.0s    

2025-10-05 15:15:55 (333 MB/s) - ‘cell_images.zip’ saved [353452851/353452851]

cell_images  cell_images.zip  sample_data


## Baseline CNN Model
Define a basic ConvNet defined with ConvLayer: Conv2D => MaxPooling2D followed by Flatten => Dense => Dense(output)

![ConvNet](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/ConvNet.png?raw=1)


# InceptionV3 Transfer Learning for Malaria Diagnosis
# Author: James Jok Dut Akuei
# Model: InceptionV3 with Transfer Learning

This notebook implements transfer learning using InceptionV3,
an efficient architecture with inception modules that parallelize
different filter sizes. InceptionV3 was chosen for:
- Lower computational cost than ResNet/VGG
- Strong performance on medical images
- Efficient multi-scale feature extraction
- Only 23.9M parameters (lighter than ResNet50's 25.6M)

Two experiments:
1. Experiment 1: Frozen base with minimal layers
2. Experiment 2: Fine-tuning with enhanced augmentation

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import pandas as pd

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("GPU:", tf.test.gpu_device_name())



TensorFlow version: 2.19.0
GPU: /device:GPU:0


# DATA PREPARATION AND SPLITTING

In [11]:
# ============================================================================
# DATA PREPARATION - Run this FIRST in new Colab
# ============================================================================

import os
import shutil
import random

# Define paths
ai_project = '.'
cell_images_dir = os.path.join(ai_project, 'cell_images')
TRAIN_DIR = os.path.join(ai_project, 'train')
TEST_DIR = os.path.join(ai_project, 'test')

# Download dataset if needed
if not os.path.exists(cell_images_dir):
    print("Downloading dataset...")
    !wget -q https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
    !unzip -q cell_images.zip
    !rm cell_images.zip
    print("✓ Dataset downloaded")

# Create train/test split
if not os.path.exists(TRAIN_DIR) or not os.path.exists(TEST_DIR):
    print("\nCreating train/test split...")

    # Create directories
    for split_dir in [TRAIN_DIR, TEST_DIR]:
        for class_name in ['Parasitized', 'Uninfected']:
            os.makedirs(os.path.join(split_dir, class_name), exist_ok=True)

    # Split data 80/20
    random.seed(42)
    for class_name in ['Parasitized', 'Uninfected']:
        src_path = os.path.join(cell_images_dir, class_name)
        images = [f for f in os.listdir(src_path) if f.endswith(('.png', '.jpg'))]
        random.shuffle(images)

        split_idx = int(len(images) * 0.8)
        train_imgs = images[:split_idx]
        test_imgs = images[split_idx:]

        print(f"{class_name}: {len(train_imgs)} train, {len(test_imgs)} test")

        # Copy to train
        for img in train_imgs:
            shutil.copy2(os.path.join(src_path, img),
                        os.path.join(TRAIN_DIR, class_name, img))

        # Copy to test
        for img in test_imgs:
            shutil.copy2(os.path.join(src_path, img),
                        os.path.join(TEST_DIR, class_name, img))

    print("\n✓ Data split complete!")

print("\nData ready for training:")
print(f"Train: {TRAIN_DIR}")
print(f"Test: {TEST_DIR}")


Creating train/test split...
Parasitized: 11023 train, 2756 test
Uninfected: 11023 train, 2756 test

✓ Data split complete!

Data ready for training:
Train: ./train
Test: ./test


# EXPERIMENT 1: Frozen Base with Minimal Layers

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT 1: InceptionV3 with Frozen Base")
print("="*80)

# Basic data augmentation
train_datagen_exp1 = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_generator_exp1 = train_datagen_exp1.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator_exp1 = train_datagen_exp1.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"\nData Summary:")
print(f"Training: {train_generator_exp1.samples}")
print(f"Validation: {val_generator_exp1.samples}")
print(f"Test: {test_generator.samples}")

# Build simplified model
def build_inceptionv3_simple():
    """
    Build InceptionV3 with minimal classification layers.
    Reduces complexity compared to typical architectures.
    """
    # Load pre-trained InceptionV3
    base_model = InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(*IMG_SIZE, 3)
    )

    # Freeze all base layers
    base_model.trainable = False

    # Minimal classification head - only 2 layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)  # Single dense layer
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

# Create model
model_exp1 = build_inceptionv3_simple()

# Compile
model_exp1.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

print("\nModel Architecture:")
total_params = model_exp1.count_params()
trainable_params = sum([tf.size(w).numpy() for w in model_exp1.trainable_weights])
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {total_params - trainable_params:,}")

# Callbacks
callbacks_exp1 = [
    EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'inceptionv3_exp1_best.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Train
print("\nTraining Experiment 1...")
history_exp1 = model_exp1.fit(
    train_generator_exp1,
    validation_data=val_generator_exp1,
    epochs=EPOCHS_EXP1,
    callbacks=callbacks_exp1,
    verbose=1
)

# Evaluate
print("\nEvaluating Experiment 1...")
test_generator.reset()
test_results = model_exp1.evaluate(test_generator, verbose=1)
test_acc_exp1, test_prec_exp1, test_rec_exp1, test_auc_exp1 = test_results[1:]

print(f"\nExperiment 1 Test Results:")
print(f"Accuracy:  {test_acc_exp1:.4f}")
print(f"Precision: {test_prec_exp1:.4f}")
print(f"Recall:    {test_rec_exp1:.4f}")
print(f"AUC:       {test_auc_exp1:.4f}")


EXPERIMENT 1: InceptionV3 with Frozen Base
Found 17638 images belonging to 2 classes.
Found 4408 images belonging to 2 classes.
Found 5512 images belonging to 2 classes.

Data Summary:
Training: 17638
Validation: 4408
Test: 5512

Model Architecture:
Total parameters: 22,065,185
Trainable parameters: 262,401
Non-trainable parameters: 21,802,784

Training Experiment 1...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 715ms/step - accuracy: 0.8134 - auc: 0.8898 - loss: 0.4047 - precision: 0.8074 - recall: 0.8244
Epoch 1: val_accuracy improved from -inf to 0.89678, saving model to inceptionv3_exp1_best.h5


552/552 ━━━━━━━━━━━━━━━━━━━━ 532s 924ms/step - accuracy: 0.8135 - auc: 0.8899 - loss: 0.4045 - precision: 0.8074 - recall: 0.8245 - val_accuracy: 0.8968 - val_auc: 0.9579 - val_loss: 0.2741 - val_precision: 0.8980 - val_recall: 0.8952 - learning_rate: 0.0010
Epoch 2/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 698ms/step - accuracy: 0.8805 - auc: 0.9463 - loss: 0.2926 - precision: 0.8699 - recall: 0.8989
Epoch 2: val_accuracy did not improve from 0.89678
552/552 ━━━━━━━━━━━━━━━━━━━━ 481s 871ms/step - accuracy: 0.8805 - auc: 0.9463 - loss: 0.2926 - precision: 0.8699 - recall: 0.8988 - val_accuracy: 0.8936 - val_auc: 0.9616 - val_loss: 0.2711 - val_precision: 0.8577 - val_recall: 0.9437 - learning_rate: 0.0010
Epoch 3/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 696ms/step - accuracy: 0.8870 - auc: 0.9515 - loss: 0.2792 - precision: 0.8701 - recall: 0.9056
Epoch 3: val_accuracy improved from 0.89678 to 0.90177, saving model to inceptionv3_exp1_best.h5


552/552 ━━━━━━━━━━━━━━━━━━━━ 480s 870ms/step - accuracy: 0.8870 - auc: 0.9515 - loss: 0.2792 - precision: 0.8701 - recall: 0.9056 - val_accuracy: 0.9018 - val_auc: 0.9634 - val_loss: 0.2484 - val_precision: 0.8694 - val_recall: 0.9456 - learning_rate: 0.0010
Epoch 4/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 696ms/step - accuracy: 0.8926 - auc: 0.9568 - loss: 0.2639 - precision: 0.8816 - recall: 0.9089
Epoch 4: val_accuracy improved from 0.90177 to 0.90313, saving model to inceptionv3_exp1_best.h5


552/552 ━━━━━━━━━━━━━━━━━━━━ 481s 872ms/step - accuracy: 0.8926 - auc: 0.9568 - loss: 0.2640 - precision: 0.8815 - recall: 0.9089 - val_accuracy: 0.9031 - val_auc: 0.9622 - val_loss: 0.2478 - val_precision: 0.8747 - val_recall: 0.9410 - learning_rate: 0.0010
Epoch 5/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 698ms/step - accuracy: 0.8971 - auc: 0.9573 - loss: 0.2610 - precision: 0.8830 - recall: 0.9137
Epoch 5: val_accuracy improved from 0.90313 to 0.90948, saving model to inceptionv3_exp1_best.h5


552/552 ━━━━━━━━━━━━━━━━━━━━ 483s 874ms/step - accuracy: 0.8971 - auc: 0.9573 - loss: 0.2610 - precision: 0.8830 - recall: 0.9137 - val_accuracy: 0.9095 - val_auc: 0.9627 - val_loss: 0.2470 - val_precision: 0.9074 - val_recall: 0.9120 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 5.

Evaluating Experiment 1...
173/173 ━━━━━━━━━━━━━━━━━━━━ 28s 161ms/step - accuracy: 0.9096 - auc: 0.4941 - loss: 0.2031 - precision: 0.3868 - recall: 0.4785

Experiment 1 Test Results:
Accuracy:  0.9236
Precision: 0.9075
Recall:    0.9434
AUC:       0.9765


# EXPERIMENT 2: Fine-tuning with Enhanced Augmentation

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT 2: InceptionV3 with Fine-tuning")
print("="*80)

# Enhanced augmentation
train_datagen_exp2 = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

# Load data
train_generator_exp2 = train_datagen_exp2.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator_exp2 = train_datagen_exp2.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# Build model with slightly more capacity for fine-tuning
def build_inceptionv3_finetune():
    """
    Build InceptionV3 with slightly more capacity.
    Still keeps architecture minimal.
    """
    base_model = InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(*IMG_SIZE, 3)
    )

    base_model.trainable = False

    # Slightly larger head for fine-tuning
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model, base_model

model_exp2, base_model_exp2 = build_inceptionv3_finetune()

# Phase 1: Train with frozen base
print("\nPhase 1: Training with frozen base...")
model_exp2.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

history_phase1 = model_exp2.fit(
    train_generator_exp2,
    validation_data=val_generator_exp2,
    epochs=EPOCHS_EXP2_PHASE1,
    callbacks=[
        EarlyStopping(monitor='val_auc', patience=3, restore_best_weights=True, mode='max')
    ],
    verbose=1
)

# Phase 2: Fine-tune top layers
print("\nPhase 2: Fine-tuning top layers...")
base_model_exp2.trainable = True

# Freeze all except last 50 layers
for layer in base_model_exp2.layers[:-50]:
    layer.trainable = False

trainable_layers = sum([layer.trainable for layer in base_model_exp2.layers])
print(f"Trainable layers in base: {trainable_layers}/{len(base_model_exp2.layers)}")

# Recompile with lower learning rate
model_exp2.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

callbacks_exp2 = [
    EarlyStopping(
        monitor='val_auc',
        patience=8,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=4,
        min_lr=1e-8,
        verbose=1
    ),
    ModelCheckpoint(
        'inceptionv3_exp2_best.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history_phase2 = model_exp2.fit(
    train_generator_exp2,
    validation_data=val_generator_exp2,
    epochs=EPOCHS_EXP2_PHASE2,
    callbacks=callbacks_exp2,
    verbose=1
)

# Combine histories
history_exp2 = {
    'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
    'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
    'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
    'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss'],
    'auc': history_phase1.history['auc'] + history_phase2.history['auc'],
    'val_auc': history_phase1.history['val_auc'] + history_phase2.history['val_auc'],
    'precision': history_phase1.history['precision'] + history_phase2.history['precision'],
    'val_precision': history_phase1.history['val_precision'] + history_phase2.history['val_precision'],
    'recall': history_phase1.history['recall'] + history_phase2.history['recall'],
    'val_recall': history_phase1.history['val_recall'] + history_phase2.history['val_recall']
}

# Evaluate
print("\nEvaluating Experiment 2...")
test_generator.reset()
test_results = model_exp2.evaluate(test_generator, verbose=1)
test_acc_exp2, test_prec_exp2, test_rec_exp2, test_auc_exp2 = test_results[1:]

print(f"\nExperiment 2 Test Results:")
print(f"Accuracy:  {test_acc_exp2:.4f}")
print(f"Precision: {test_prec_exp2:.4f}")
print(f"Recall:    {test_rec_exp2:.4f}")
print(f"AUC:       {test_auc_exp2:.4f}")


EXPERIMENT 2: InceptionV3 with Fine-tuning
Found 17638 images belonging to 2 classes.
Found 4408 images belonging to 2 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Phase 1: Training with frozen base...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 577s 1s/step - accuracy: 0.7448 - auc: 0.8202 - loss: 0.5145 - precision: 0.7399 - recall: 0.7442 - val_accuracy: 0.8755 - val_auc: 0.9378 - val_loss: 0.3238 - val_precision: 0.8475 - val_recall: 0.9156
Epoch 2/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 526s 954ms/step - accuracy: 0.8456 - auc: 0.9190 - loss: 0.3588 - precision: 0.8284 - recall: 0.8718 - val_accuracy: 0.8662 - val_auc: 0.9401 - val_loss: 0.3585 - val_precision: 0.8895 - val_recall: 0.8362
Epoch 3/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 524s 950ms/step - accuracy: 0.8498 - auc: 0.9253 - loss: 0.3453 - precision: 0.8346 - recall: 0.8761 - val_accuracy: 0.8723 - val_auc: 0.9375 - val_loss: 0.3205 - val_precision: 0.8224 - val_recall: 0.9496
Epoch 4/5
552/552 ━━━━━━━━━━━━━━━━━━━━ 540s 979ms/step - accuracy: 0.8528 - auc: 0.9292 - loss: 0.3342 - precision: 0.8307 - recall: 0.8833 - val_accuracy: 0.8838 - val_auc: 0.9404 - val_loss: 0.3113 - val_precision: 0.8384 - val_recall: 0.9510
Epoch 5/5
552/552 ━━━━━

552/552 ━━━━━━━━━━━━━━━━━━━━ 600s 1s/step - accuracy: 0.8945 - auc: 0.9490 - loss: 0.2820 - precision: 0.8764 - recall: 0.9191 - val_accuracy: 0.9215 - val_auc: 0.9633 - val_loss: 0.2255 - val_precision: 0.8940 - val_recall: 0.9564 - learning_rate: 1.0000e-04
Epoch 2/5
286/552 ━━━━━━━━━━━━━━━━━━━━ 3:25 774ms/step - accuracy: 0.9285 - auc: 0.9712 - loss: 0.2006 - precision: 0.9045 - recall: 0.9598

# VISUALIZATIONS

In [ ]:
print("\n" + "="*80)
print("GENERATING VISUALIZATIONS")
print("="*80)

# Get predictions
test_generator.reset()
y_true = test_generator.classes
y_pred_prob_exp1 = model_exp1.predict(test_generator, verbose=0).flatten()
y_pred_exp1 = (y_pred_prob_exp1 > 0.5).astype(int)

test_generator.reset()
y_pred_prob_exp2 = model_exp2.predict(test_generator, verbose=0).flatten()
y_pred_exp2 = (y_pred_prob_exp2 > 0.5).astype(int)

# Learning curves
def plot_learning_curves(history, title, exp_num):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    axes[0].plot(history['accuracy'], label='Training', linewidth=2)
    axes[0].plot(history['val_accuracy'], label='Validation', linewidth=2)
    axes[0].set_title(f'{title} - Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(alpha=0.3)

    axes[1].plot(history['loss'], label='Training', linewidth=2)
    axes[1].plot(history['val_loss'], label='Validation', linewidth=2)
    axes[1].set_title(f'{title} - Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'inceptionv3_exp{exp_num}_curves.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_learning_curves(history_exp1.history, 'Experiment 1: Frozen Base', 1)
plot_learning_curves(history_exp2, 'Experiment 2: Fine-tuning', 2)

# Confusion matrices
def plot_confusion_matrix(y_true, y_pred, title, exp_num):
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Parasitized', 'Uninfected'],
                yticklabels=['Parasitized', 'Uninfected'])
    plt.title(f'{title}\nConfusion Matrix', fontsize=14, fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'inceptionv3_exp{exp_num}_cm.png', dpi=300, bbox_inches='tight')
    plt.show()
    return cm

cm_exp1 = plot_confusion_matrix(y_true, y_pred_exp1, 'Experiment 1: Frozen Base', 1)
cm_exp2 = plot_confusion_matrix(y_true, y_pred_exp2, 'Experiment 2: Fine-tuning', 2)

# ROC curves
def plot_roc_curve(y_true, y_pred_proba, title, exp_num):
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{title}\nROC Curve', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'inceptionv3_exp{exp_num}_roc.png', dpi=300, bbox_inches='tight')
    plt.show()
    return roc_auc

roc_auc_exp1 = plot_roc_curve(y_true, y_pred_prob_exp1, 'Experiment 1: Frozen Base', 1)
roc_auc_exp2 = plot_roc_curve(y_true, y_pred_prob_exp2, 'Experiment 2: Fine-tuning', 2)


GENERATING VISUALIZATIONS


NameError: name 'test_generator' is not defined